<a href="https://colab.research.google.com/github/torzdf/faceswap/blob/colab/faceswap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left">
  <a href="https://faceswap.dev"><img src="https://i.imgur.com/zHvjHnb.png"></img></a>Welcome to the FaceSwap Notebook
</p>



The FaceSwap Notebook is a means of running FaceSwap on Google Colab.

Colab will give you a free GPU for up to a 12 hour period (as long as your browser window remains open).

Please make sure you understand Google Colaboratory's terms of service. This notebook is for testing out Faceswap and experimenting. It is not for creating swaps on an industrial scale. [See here for more info](https://research.google.com/colaboratory/faq.html#gpu-availability).

At a minimum you must run the [Prerequisites](#scrollTo=GVzyxw4xkKI7). Once complete, you can select the task that you wish to run.

**NB:** This notebook is provided as a courtesy. It is not a priority for the FaceSwap project so there is no guarantee that it will remain up to date and support is likely to be limited. Please be aware that this Notebook lacks some features from the full application.

>[Prerequisites](#scrollTo=GVzyxw4xkKI7)

>>[GPU Check](#scrollTo=_jYRKp5zkYMU)

>>[Link Google Drive](#scrollTo=Sb1aWvH3phjA)

>>[Get FaceSwap](#scrollTo=oql-3Tf7lquV)

>[Set up FaceSwap](#scrollTo=h9DWTBSHOwxO)

>[Execute FaceSwap Task](#scrollTo=ffaXw9B5LHNg)



# Prerequisites
First up we need to make sure we have a GPU instance and set up our environment.

Whatever task you plan to perform, you must run these steps first.

## GPU Check
Run the following code block to set up the initial helper functions and make sure you have been allocated a GPU:

In [0]:
#@title
import os
from ipywidgets import widgets, Layout
# SET GLOBALS
_HOMEDIR = "/content"
if not os.path.exists(_HOMEDIR):
  os.mkdir(_HOMEDIR)
_GDRIVE= os.path.join(_HOMEDIR, "gdrive", "My Drive")
_LOC_SETTINGS=os.path.join(_GDRIVE, "_faceswap_colab")
os.chdir(_HOMEDIR)
# HELPER FUNCTIONS
def html_color(text, color):
  """ Return given text in a span for given color """
  colors = dict(primary="--colab-primary-text-color",
                blue="--ansi-blue",
                green="--ansi-green",
                red="--ansi-red",
                yellow="--ansi-yellow")
  span="<span style='color:var({});'>".format(colors[color.lower()])
  return "{}{}</span>".format(span, text)
def html_show(html, kwargs=None):
  """ Create HTML Widget and display it """
  kwargs = kwargs if isinstance(kwargs, dict) else dict()
  output = widgets.HTML(value=html, **kwargs)
  display(output)
def write_backend_config(config_file):
  """ Set the backend config to "nvidia" for the given location """
  if not os.path.isdir(os.path.dirname(config_file)):
    os.mkdir(os.path.dirname(config_file))
  with open(config_file, "w") as fconfig:
    fconfig.write("{\"backend\": \"nvidia\"}")
# GPU CHECK
!nvidia-smi

 You should receive output similar to below:
```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+
```

If you have not been allocated a GPU then select `Runtime` > `Change runtime type` from the top menu and ensure that the `Runtime type` is `Python 3` and `Hardware Acceleration` is `GPU`

## Link Google Drive
Now we know we have a GPU instance, we need to link your Google Drive.

This Notebook will create a folder on your Google Drive (if it doesn't exist) called `_faceswap_colab`. This is where various utility and options files will be stored so that you can re-use them in subsequent sessions.

You will need to give Colab permissions to mount your drive, so run the code block below and follow the instructions to mount your drive:

In [0]:
#@title
from google.colab import drive
def _import_gdrive():
  drive.mount(os.path.dirname(_GDRIVE), force_remount=True)
  # Create SETTINGS location if it doesn't exist
  if not os.path.isdir(_LOC_SETTINGS):
    html_show(html_color("Creating folder '{}'".format(_LOC_SETTINGS), "green"))
    os.makedirs(_ROOT)

  msg = html_color("<strong>Complete.</strong> ", "green")
  msg += html_color("<strong>NB:</strong> If you received `shell-init` "
                    "errors then select `Runtime` > `Restart Runtime...` from "
                    "the main menu and try again.", "primary")
  html_show(msg)
_import_gdrive()

## Get FaceSwap
Time to tell Colab where to find/install the FaceSwap App. Select your options in the form, then hit play:

In [0]:
#@title App install location
#@markdown Where do you want Faceswap installed?
#@markdown * GoogleDrive: [Recommended] - Store the FaceSwap App on your Google
#@markdown Drive. This means any config files that are changed won't get lost
#@markdown when the instance destroys. However it will take up about 200MB of
#@markdown space for the base app and several GBs for the associated models.
#@markdown * Local: FaceSwap App will be stored in the Colab Notebook. The app
#@markdown will disappear when the instance dies.
location = "Local"  #@param ['GoogleDrive', 'Local']
#@markdown Do You want to update an existing app or create a new one?
#@markdown * Update: [Recommended] - If the FaceSwap App is already installed in
#@markdown the chosen location, it will be updated. If the app doesn't already
#@markdown exist, it will be installed.
#@markdown * New: Uninstall any installed FaceSwap App and Reinstall for the
#@markdown chosen location.
app_update = "New"  #@param ['New', 'Update']

import shutil
import sys
_APPDIR=_LOC_SETTINGS if location == "GoogleDrive" else _HOMEDIR
_APPDIR=os.path.join(_APPDIR, "faceswap")
_APP=os.path.join(_APPDIR, "faceswap.py")

def _get_repo():
  """ Create or update the Repo """
  html_show(html_color("App Location: {}".format(_APP), "blue"))
  if app_update == "New" or not os.path.isfile(_APP):
    if os.path.isdir(_APPDIR):
      html_show(html_color("Uninstalling pre-existing app...", "green"))
      shutil.rmtree(_APPDIR)
    html_show(html_color("Getting Faceswap...", "green"))
    !git clone --depth 1 -b master https://github.com/deepfakes/faceswap.git "$_APPDIR"
    os.chdir(_APPDIR)
  else:
    os.chdir(_APPDIR)
    html_show(html_color("Updating App...", "green"))
    !git pull 2>&1
def _update_dependencies():
  """ Update Faceswap Dependencies """
  html_show(html_color("Setting Tensorfow to version 1.x...", "green"))
  %tensorflow_version 1.x
  html_show(html_color("Setting backend to nvidia...", "green"))
  write_backend_config(os.path.join(_APPDIR, "config", ".faceswap"))
  html_show(html_color(
      "Updating Dependencies. This may take a minute or two...", "green"))
  !python "$_APPDIR/update_deps.py"
  html_show(html_color("<strong>Faceswap Ready!</strong>", "green"))
  # Dummy in a backend config in root python dir to prevent backend selector
  # poppping when importing directly from FaceSwap
  sys_location = os.path.dirname(os.path.realpath(sys.argv[0]))
  sys_conf = os.path.join(sys_location, "config", ".faceswap")
  write_backend_config(sys_conf)
try:
  _get_repo()
  _update_dependencies()
except OSError as err:
  print("\033[31mYour GDrive could not be reached. Select `Runtime` > `"
      "Restart Runtime... from the main menu and go back to the previous "
      "cell (Link Google Drive)")

# Set up FaceSwap
Now we're all setup, it's time to select a task and set it up.

**NB:** You can edit plugin configuration files for each task from this location: `/drive/My Drive/_faceswap_colab/faceswap/config`.
<br /><br />
First up, let's decide which task we want to run. Select the task then run the code block to generate the task options form:

In [0]:
#@title Configure FaceSwap Task
#@markdown Which task do you want to run?
task = "Extract"  #@param ['Extract', 'Train', 'Convert']

#@markdown **NB:** You can copy file paths from the `Files` tab in the lefthand
#@markdown panel by right clicking the desired file and selecting `Copy path`.
#@markdown Your GoogleDrive is located in `/drive/My Drive`.

#@markdown For an in depth look into the options, you can check out the guides
#@markdown over at [Faceswap.dev](https://forum.faceswap.dev/app.php/tag/Guide?sid=127ccba54e6de01f5fc4378b2b88a151).

import json
import inspect
from functools import partial

def _get_cliopts(section):
  """ obtain the cli options for the given section """
  # Items to not be displayed
  skip_list=("logfile", "gui", "singleprocess", "colab", "write-image", "gpus",
             "ping-pong", "allow-growth")
  skip_prefix=("preview", "timelapse")
  
  # Items to cache data locally
  cache_prefix = ("input", "output")
  cache_suffix = ("-dir", "-a", "-b")
  
  browser_lookup = dict(DirFullPaths="Folder",
                        FileFullPaths="File",
                        DirOrFileFullPaths="Folder or a File")

  args = _get_task(section)
  final_args = []
  for arg in args:
    opt_name = arg["opts"][-1].replace("--", "")
    if (opt_name in skip_list or opt_name.startswith(skip_prefix)):
      continue

    arg["cache"] = (opt_name.lower().startswith(cache_prefix) and
                    opt_name.lower().endswith(cache_suffix))

    val = arg.get("default", None)
    _title = arg["opts"][-1][2:] if len(arg["opts"]) == 2 else arg["opts"][-1][1:]
    arg["title"] = _title.replace("-", " ").replace("_", " ").title()

    choices = arg.get("choices", [])
    action = arg.get("action", None)
    action = action.__name__ if inspect.isclass(action) else action
    action = None if action == "Radio" and len(choices) > 3 else action
    if action == "Radio":
      widge = partial(widgets.RadioButtons, options=choices)
    elif action == "Slider" and arg["type"] == int:
      widge = partial(widgets.IntSlider,
                      min=arg["min_max"][0],
                      max=arg["min_max"][1],
                      step=arg["rounding"],
                      continuous_update=False,
                      orientation='horizontal',
                      readout=True,
                      readout_format='d')
    elif action == "Slider" and arg["type"] == float:
      widge = partial(widgets.FloatSlider,
                      min=arg["min_max"][0],
                      max=arg["min_max"][1],
                      step=arg["rounding"] / 100,
                      continuous_update=False,
                      orientation='horizontal',
                      readout=True,
                      readout_format='.1f')
    elif action == "store_true":
      widge = widgets.Checkbox
    elif action in browser_lookup:
        widge = partial(
            widgets.Text,
            placeholder='Enter the path to a {}'.format(browser_lookup[action]))
    elif choices:
      widge = partial(widgets.Dropdown, options=choices)
    elif arg.get("type", str) == int:
      widge = widgets.IntText
    elif arg.get("type", str) == float:
      widge = widgets.FloatText
    else:
      widge = partial(widgets.Text, placeholder='Type something')
    arg["control"] = widge(value=val, disabled=False, layout=Layout(width="15%"))
    final_args.append(arg)
  return final_args

def _get_task(section):
  """ Return the FaceSwap options for a section from lib.cli """
  import lib.cli as cli
  mod_class=[name for name, obj in inspect.getmembers(cli)
             if inspect.isclass(obj) and name.lower().endswith("args")
             and name.lower().startswith(section)
             and name.lower() != "extractconvertargs"][0]
  
  name = mod_class.lower().replace("args", "")
  meth = getattr(cli, mod_class)(None, name)
  args = meth.argument_list + meth.optional_arguments + meth.global_arguments
  
  # Print the Form Header
  html_show("<h1>{}</h1>".format(name.title()))
  for idx, line in enumerate(meth.info.split("\n")):
    line = "<h4>{}</h4>".format(line) if idx == 0 else line
    if "plugins can be configured" in line:
      conf_file = os.path.join(_APPDIR, "config", "{}.ini".format(name.lower()))
      line = "{} plugins can be configured in: {}".format(name.title(),
          html_color(conf_file, "blue"))
    html_show(html_color(line, "primary"))  

  return args

# Global the current cli options
_CLI_OPTS = _get_cliopts(task.lower())

def _display_form():
  """ Display the cli options in a useful form """
  groups = dict()
  for opt in _CLI_OPTS:
    groups.setdefault(opt.get("group", "").title(), []).append(opt)

  for group, opts in groups.items():
    html_show("<h2>{}</h2>".format(group))
    for opt in opts:
      opt_lbl = widgets.HTML(value=html_color(opt["title"], "blue"),
                             layout=Layout(width='10%'))
      hlp = opt.get("help", "")
      if hlp.startswith("R|"):
        lines = hlp[2:].split("L|")
        hlp_text = (" ".join("<li>{}</li>".format(html_color(row, "primary"))
                    for row in lines[1:]))
        hlp_text = "{}<ul>{}</ul>".format(html_color(lines[0], "primary"),
                                          hlp_text)
      else:
        hlp_text = html_color(hlp, "primary")
      hlp_text = widgets.HTML(value=hlp_text, layout=Layout(width='73%'))
      spacer = widgets.HTML(value="", layout=Layout(width='2%'))
      opt_ctrl = widgets.HBox((opt_lbl, opt["control"], spacer, hlp_text))
      display(opt_ctrl)

      
_display_form()

In [0]:
#@title Load or Save Task Options
#@markdown Run this cell to get saving and loading options. You should re-run
#@markdown this cell again if you change tasks.
from IPython.display import clear_output
# File handling
def load_tasks():
  def _load(b):
    fname = os.path.join(load_path, selections.value)
    with open(fname, "rb") as infile:
      opts = json.loads(infile.read().decode("utf-8"))
      opts = opts[task.lower()]
    for arg in _CLI_OPTS:
      title = arg["title"]
      if title not in opts:
        continue
      arg["control"].value = opts[title]
    clear_output()
    print("\033[0mLoaded options from: \033[34m{}\033[0m".format(fname))
    print("Re-run this cell to bring back Option loading/saving dialogue")

  load_path = os.path.join(_LOC_SETTINGS, "tasks", task.lower())
  info = widgets.HTML(value=html_color(
      "<strong>Load a saved {} Task</strong>".format(task), "blue"))
  if not os.path.exists(load_path):
    _err = widgets.HTML(value="No {} Task files exist.".format(task))
    return widgets.VBox((info, _err))
  fnames = [f for f in os.listdir(load_path)
            if os.path.splitext(f)[-1] == ".fst"]
  if not fnames:
    _err = widgets.HTML(value="No {} Task files exist.".format(task))
    return widgets.VBox((info, _err))
  selections = widgets.RadioButtons(
    options=fnames,
    value=fnames[0],
    disabled=False)
  do_load = widgets.Button(description="Load")
  do_load.on_click(_load)
  return widgets.VBox((info, selections, do_load))
    
def save_tasks():
  def _save(b):
    fname = save_name.value
    if not fname:
      html_show(html_color("Error: No save name provided", "red"))
      return
    save_path = os.path.join(_LOC_SETTINGS, "tasks", task.lower())
    if not os.path.isdir(save_path):
      os.makedirs(save_path)
    fname = fname[:-4] if fname.endswith(".fst") else fname
    save_file = os.path.join(save_path, "{}.fst".format(fname))
    save_opts = {task.lower(): {opt["title"]: opt["control"].value
                                 for opt in _CLI_OPTS},
                  "tab_name": task.lower()}
    serialized = json.dumps(save_opts, indent=2).encode("utf-8")

    with open(save_file, "wb") as out_file:
      out_file.write(serialized)
    clear_output()
    print("\033[Options saved to: \033[34m{}\033[0m".format(save_file))
    print("Re-run this cell to bring back Option loading/saving dialogue")
  
  info = widgets.HTML(value=html_color(
      "<strong>Save the current {} Task</strong>".format(task), "blue"))
  lbl_name = widgets.HTML(value=html_color("Name ", "primary"))
  save_name = widgets.Text(
    placeholder='Enter the name for these options')
  box_name = widgets.VBox((lbl_name, save_name))
  lbl_save = widgets.HTML(value="<br />")
  do_save = widgets.Button(description="Save")
  do_save.on_click(_save)
  box_save = widgets.VBox((lbl_save, do_save))
  save_box = widgets.HBox((box_name, box_save))
  return widgets.VBox((info, save_box))
load_section = load_tasks()
spacer = widgets.HTML(value="&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;")
save_section = save_tasks()
display(widgets.HBox((load_section, spacer, save_section)))

# Execute FaceSwap Task
If you're happy with your options, then it's time to run the Task. Run the codeblock below to generate the task:

In [0]:
#@title
# Global the cache mapping
_CACHE_DIR = "/content/cache"

def _del_cache():
  if os.path.exists(_CACHE_DIR):
    shutil.rmtree(_CACHE_DIR)
    os.makedirs(_CACHE_DIR)

def _copy_files(ttl, src, dst):
    src_display = html_color(src, "blue")
    dst_display = html_color(dst, "blue")
    html_show(html_color("Copying {} from {} to {}...".format(ttl,
                                                              src_display,
                                                              dst_display),
                          "primary"))
    if os.path.isdir(src) and not os.path.exists(dst):
      os.makedirs(dst)
    elif not os.path.exists(os.path.dirname(dst)):
      os.makedirs(os.path.dirname(dst))

    if os.path.isdir(src):
      for fname in os.listdir(src):
        shutil.copyfile(os.path.join(src, fname), os.path.join(dst, fname))
    else:
      shutil.copyfile(src, dst)

def _get_args():
  """ Generate cli arguments and display info """
  
  exe_args = [task.lower()]
  for opt in _CLI_OPTS:
    val = opt["control"].value
    if not val:
      continue

    # Track cache mapping
    if opt["cache"] and val.startswith(_GDRIVE):
      opt["mapping"] = val.replace(_GDRIVE, _CACHE_DIR)

    exe_args.append(opt["opts"][0])

    if opt.get("action", None) != "store_true":
      val = "\"{}\"".format(val) if isinstance(val, str) and " " in val else val
      exe_args.append(val)

  if task == "Train":
    exe_args.append("-w")
    info = "{}{}".format(html_color("Training preview will be available to "
                                    "view at: ", "primary"),
                        html_color(os.path.join(_APPDIR, "training_preview.jpg"),
                                    "blue"))
    html_show(info)
  info = "{}{}".format(html_color("The command to be run is as follows: ",
                                  "primary"),
                      html_color(" ".join(str(a) for a in exe_args), "blue"))
  html_show(info)
  exe_args.append("--colab")
  return exe_args

def _cache_files():
  """ Copy any remote files to local """
  html_show(html_color("<strong>Caching Data...<strong>", "green"))
  _del_cache()
  cache_args = {opt["opts"][0]: dict(opt_name=opt["title"],
                                     src=opt["control"].value,
                                     dst=opt["mapping"])
                 for opt in _CLI_OPTS if opt.get("mapping", None) is not None}

  if not cache_args:
    html_show(html_color("No data to cache", "blue"))
    return False

  for args in cache_args.values():
    if args["opt_name"].lower().startswith("input"):
      _copy_files(args["opt_name"], args["src"], args["dst"])
  return cache_args

def rewrite_exe_args(cache_args, exe_args):
  """ Rewrite the command line params to point at cache location """
  if not cache_args:
    return exe_args
  for idx, key in enumerate(exe_args):
    if key in cache_args:
      old_location = exe_args[idx + 1]
      exe_args[idx + 1] = old_location.replace(cache_args[key]["src"],
                                               cache_args[key]["dst"])
  return exe_args

def _clean_up(cache_args):
  """ Transfer cached information back to GDrive and remove cache """
  if not cache_args:
    _del_cache()
    return
  html_show(html_color("<strong>Copying Data from Cache...<strong>", "green"))
  for args in cache_args.values():
    ttl = args["opt_name"]
    src = args["dst"]
    dst = args["src"]
    if ttl.lower().startswith("output"):
      _copy_files(ttl, src, dst)
    if ttl.lower().startswith("input") and task.lower() == "extract":
      srcdirname = src if os.path.isdir(src) else os.path.dirname(src)
      dstdirname = dst if os.path.isdir(dst) else os.path.dirname(dst)
      alignments_files = [fname
                          for fname in os.listdir(srcdirname)
                          if os.path.splitext(fname)[-1] == ".fsa"]
      for algn in alignments_files:
        _copy_files(ttl,
                    os.path.join(srcdirname, algn),
                    os.path.join(dstdirname, algn))
  _del_cache()

def _wait_on_user():
  def button_clicked(b):
    cached_files = _cache_files()
    cmd_opts = rewrite_exe_args(cached_files, exe_args)
    html_show(html_color("<strong>Executing FaceSwap...</strong>", "green"))
    cmd_opts = " ".join(str(a) for a in cmd_opts)
    !python faceswap.py $cmd_opts
    _clean_up(cached_files)
    html_show(html_color("<strong>{} Task Complete!</strong>".format(
        task.title()), "green"))

  exe_args = _get_args()
  button = widgets.Button(description="Execute {}!".format(task))
  button.on_click(button_clicked)
  display(button)

_wait_on_user()
  
  